In [1]:
"""
基于 Graph API 的评估器——优化器工作流实现
"""

from typing_extensions import TypedDict, Literal
from pydantic import BaseModel, Field

from langgraph.graph import StateGraph, START, END

from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

llm = ChatOpenAI(model="Qwen/Qwen2.5-7B-Instruct")


class Feedback(BaseModel):
    grade: Literal["funny", "not funny"] = Field(description="判断笑话是否有趣。")
    feedback: str = Field(description="如果笑话不好笑，那么提供改进它的反馈。")


evaluator = llm.with_structured_output(Feedback)


class State(TypedDict):
    joke: str
    topic: str
    feedback: str
    funny_or_not: str


def llm_call_generator(state: State):
    if state.get("feedback"):
        msg = llm.invoke(
            f"写一个关于{state['topic']}的笑话，但要考虑反馈：{state['feedback']}"
        )
    else:
        msg = llm.invoke(f"写一个关于{state['topic']}的笑话")
    return {"joke": msg.content}


def llm_call_evaluator(state: State):
    grade = evaluator.invoke(f"评估笑话{state['joke']}")
    return {"funny_or_not": grade.grade, "feedback": grade.feedback}


def route_joke(state: State):
    if state["funny_or_not"] == "funny":
        return "Accepted"
    elif state["funny_or_not"] == "not funny":
        return "Rejected + Feedback"


optimizer_builder = StateGraph(State)

optimizer_builder.add_node("llm_call_generator", llm_call_generator)
optimizer_builder.add_node("llm_call_evaluator", llm_call_evaluator)

optimizer_builder.add_edge(START, "llm_call_generator")
optimizer_builder.add_edge("llm_call_generator", "llm_call_evaluator")
optimizer_builder.add_conditional_edges(
    "llm_call_evaluator",
    route_joke,
    {"Accepted": END, "Rejected + Feedback": "llm_call_generator"},
)

optimizer_builder = optimizer_builder.compile()

state = optimizer_builder.invoke({"topic": "cats"})
print(state["joke"])

当然可以！这里有一个关于猫的轻松笑话：

为什么猫不喜欢上网？

因为它们只喜欢“猫”网，而不爱“汪”网！
